# Telegram  chat-bot

* It is the very easy example of telegram chat-bot. 

* To make it more complicated and clever you need to modify `gen_answer` function

* Subscribe to my channel [@AbstractDL](https://t.me/abstractDL), it is about Deep Learning ;)

In [ ]:
!pip install python-telegram-bot
import telegram
from telegram.ext import Updater, CommandHandler
from telegram.ext import MessageHandler, Filters

from telegram import Update
from telegram.ext import CallbackContext

import regex as re
import glob
import os.path
from IPython.display import clear_output
clear_output()

### API Token
Here you need to insert your API token. 

You can create a bot and obtain it token here https://t.me/BotFather. It will take about 30 sec (just creating name)

In [ ]:
TOKEN = "PUT HERE YOUR API BOT TOKEN"

### Generate answer
It is a fuction, that creates the response of the bot given the lis of all messages. If you wand to make a generative chit-chat bot you can place something like [ruGPT-3](https://github.com/sberbank-ai/ru-gpts) here.


In [ ]:
def gen_answer(history):
    """
    Function for generating answer, given dialog history.
    The last message in the history is the current message.
    """
    answer = f"You history is {history}"
    return answer

### Хэндлеры
Тут объявляются базовые функции бота, такие как `start`, `help`, `chat`

In [ ]:
updater = Updater(token=TOKEN, use_context=True)
dispatcher = updater.dispatcher
user_history = {}
users = {}

HELLO = "This chatbot sends your message history in replay\n\n_Создатель: @AbstractDL_"
HELLO_START = "It is a starting message"

def help(update: Update, context: CallbackContext):
    """
    This happens when the user taps "help" button or sends "\help"
    You can add "help" button in the bot menu in the https://t.me/BotFather
    """
    chat_id=update.effective_chat.id
    text = HELLO
    context.bot.send_message(chat_id=chat_id, text=text, parse_mode="Markdown")


def start(update: Update, context: CallbackContext):
    """
    This happens whe the user taps "start" button or sends "\start"
    """
    chat_id=update.effective_chat.id
    text = HELLO_START
    context.bot.send_message(chat_id=chat_id, text=text, parse_mode="Markdown")

    first_name = update.message.chat.first_name
    last_name = update.message.chat.last_name
    username = update.message.chat.username

    users[chat_id] = (username, first_name, last_name)
    print(f"#############\nNEW_USER:     {chat_id}{users[chat_id]}\n#############")

    
def chat(update: Update, context: CallbackContext):
    """
    Main chat handler
    """
    chat_id=update.effective_chat.id
    updater.bot.send_chat_action(chat_id=chat_id, action=telegram.ChatAction.TYPING)
    message = update.message.text
    message = re.sub("^a-zA-Zа-яА-Я0-9.,!?/@", '', message)
    
    chat_id=update.effective_chat.id
    first_name = update.message.chat.first_name
    last_name = update.message.chat.last_name
    username = update.message.chat.username
    
    if not chat_id in user_history:
        user_history[chat_id] = [message]
    else:
        user_history[chat_id].append(message)
        
    history = user_history[chat_id]
    answer = gen_answer(history)
    user_history[chat_id].append(answer)
    context.bot.send_message(chat_id=chat_id, text=answer)
    print(f"{chat_id}:\t{message} ||| {answer}")
        

chat_handler = MessageHandler(Filters.text & (~Filters.command), chat)
dispatcher.add_handler(chat_handler)

help_handler = CommandHandler("help", help, filters=Filters.command)
dispatcher.add_handler(help_handler)

start_handler = CommandHandler("start", start, filters=Filters.command)
dispatcher.add_handler(start_handler)

### Запуск бота в бесконечном цикле

In [ ]:
updater.start_polling()
updater.idle()